In [47]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


To construct a graph that is light weight enough while still being able to generate insights, the input data will be split into 3 groups: Categorical values to create subgroups, numerical values to act as idenfitifiers to identify the nearest neighbours for each subgroup and the resale price which is the target variable.

The first step is clean the numeric values that will be used for the k-nn algo.

In [48]:
data_df = pd.read_csv('data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

In [49]:
listing_df = data_df.copy()
# splitting the month and the year allows to check long term trend as well as seasonality in the year itself.
listing_df[['year', 'month_2']] = listing_df['month'].str.split('-').to_list()
listing_df['year'] = listing_df['year'].astype(int)
listing_df['month_2'] = listing_df['month_2'].astype(int)

listing_df['month_index'] = listing_df['year'].astype(int)*12 + listing_df['month_2'].astype(int)
listing_df['lease_commence_index'] = listing_df['lease_commence_date']*12

# recalculate remaining_lease in years, rounding down to the closest year
listing_df['remaining_lease'] = np.floor((99 * 12 - (listing_df['month_index'] - listing_df['lease_commence_index']))/12)

listing_df['flat_model'] = listing_df.flat_model.replace(
listing_df.groupby('flat_model').agg({'resale_price':'mean', 'flat_model':'count'}).rename(columns={'flat_model':'count'}).reset_index().query("count < 500")['flat_model'].to_list(),
'OTHERS'
)
remaining_lease_df = listing_df[['remaining_lease']]


listing_df = listing_df.drop(columns=['month', 'block', 'year', 'month_2', 'lease_commence_index'])

Categorical values will also be one-hot encoded to aid in the k-nn algorithm

In [50]:
cat_cols = list(listing_df.select_dtypes(exclude='number').columns)
for column in ['flat_type', 'flat_model', 'storey_range']:
    listing_df = pd.concat([listing_df, pd.get_dummies(listing_df[[column]])], axis=1)

The numeric values generated in the previous step are scaled to reduce the impact of high variance categories vs low-variance categories.

In [51]:
for column in listing_df.select_dtypes(include='number').columns:
    if column != 'month_index':
        listing_df[column] = StandardScaler().fit_transform(listing_df[[column]])


The graph is initialised with values that could inform the resale price

In [52]:
G = nx.Graph()
for i, row in listing_df.iterrows():
    G.add_node(i,
                flat_type=row['flat_type'],
                flat_model=row['flat_model'],
                town=row['town'],
                street_name=row['street_name'],
                storey_range=row['storey_range'],
                remaining_lease=remaining_lease_df.iloc[i],
                resale_price=row['resale_price'])

All the listings are grouped over the categorical columns and edges are added between the k-closest values. The other limitation that's applied is to only look at the 12 months around the node being considered. This limits the number of edges, while ensuring that the most relevant are kept.

In [53]:
for column in cat_cols:
    for col_name, group in listing_df.groupby(column):

        subrows = group.index

        k = min(5,len(group)-1)

        nn = NearestNeighbors(n_neighbors=k+1)
        nn.fit(group.drop(columns=cat_cols+['resale_price', 'month_index']))
        distances, indices = nn.kneighbors(group.drop(columns=cat_cols+['resale_price', 'month_index']))

        for row_pos, (d, i) in enumerate(zip(distances, indices)):
            u = subrows[row_pos]

            for dist, nbr_pos in zip(d, i):
                if nbr_pos != row_pos:
                    v = subrows[nbr_pos]

                    w = float(np.exp(-dist))

                    if G.has_edge(u, v):
                        if w > G[u][v]['weight']:
                            G[u][v]['weight'] = w
                    else:
                        G.add_edge(u,v,weight=w)

While the average degree is 15.3, the median is 14, implying that the majority of the nodes have at least some edges, and the p90 of 21 shows that there are very few nodes which are too heavily connected. The low numnber of isolated nodes means that every node will contribute, and helps in analysing the data.

The assortavity shows that people care more about the town as compared to the street_name, and are more inclined to consider the type of house more, i.e. flat_type, flat_model and storey_range.

In [54]:
num_nodes, num_edges = G.number_of_nodes(), G.number_of_edges()
print(f"Graph: {num_nodes} nodes, {num_edges} edges")

deg = np.array([d for _, d in G.degree()])
print("Degree: mean", deg.mean(), "median", np.median(deg), "p90", np.percentile(deg, 90))

trans = nx.transitivity(G)
print("Global clustering (transitivity):", round(trans, 4))

for column in cat_cols + ['storey_range']:
    print(f"Assortativity by {column}:", round(nx.attribute_assortativity_coefficient(G, column), 3))


Graph: 80374 nodes, 616276 edges
Degree: mean 15.335207903053226 median 14.0 p90 21.0
Global clustering (transitivity): 0.3938
Assortativity by town: 0.632
Assortativity by flat_type: 0.997
Assortativity by street_name: 0.42
Assortativity by storey_range: 0.93
Assortativity by flat_model: 0.993
Assortativity by storey_range: 0.93


TASK II

The Louvain community allows grouping of nodes into communities. This is done by trying to maximise modularity, which captures how much more densely nodes in a community are connected to each other, compared to random nodes.

In [55]:
import community

In [56]:
partition = community.best_partition(G, weight='weight', resolution=1.0)  # dict: node -> community_id
nx.set_node_attributes(G, partition, 'community')

In [57]:
part_df = pd.DataFrame({'community': list(partition.values())})
data_df = pd.concat([data_df, part_df], axis=1)

In [58]:
top_3_communities = pd.DataFrame(data_df.community.value_counts().head(3)).reset_index()

Extracting the 3 biggest communities and inspecting the profile of the top 3 combinations of town, flat_type and storey_range shows us similarities, for example, the biggest community is mostly 3 rooms between the 1st to 3rd floor, meanwhile, the second biggest community is primarily composed of executive flat and the 3rd biggest is primarily 3-rooms in Punggol.

In [59]:
summary = pd.DataFrame(data_df.merge(top_3_communities, on= 'community').groupby(['community', 'town', 'flat_type', 'storey_range'])['community'].count()).rename(columns={'community':'count'}).reset_index()
summary.sort_values('count', ascending=False).groupby('community').head(3)

,community,town,flat_type,storey_range,count
145,48,TAMPINES,4 ROOM,01 TO 03,282
140,48,JURONG WEST,4 ROOM,01 TO 03,225
142,48,PASIR RIS,4 ROOM,01 TO 03,201
77,37,PASIR RIS,EXECUTIVE,04 TO 06,136
78,37,PASIR RIS,EXECUTIVE,07 TO 09,128
119,37,WOODLANDS,EXECUTIVE,07 TO 09,116
262,59,PUNGGOL,3 ROOM,10 TO 12,82
263,59,PUNGGOL,3 ROOM,13 TO 15,70
261,59,PUNGGOL,3 ROOM,07 TO 09,61


Task III

The last part of this exercise is to train a GNN to predict resale price, based on the graph network that's been created in the previous steps.

To ensure that this training works on as many devices as possible, while using the available resources, the device checks if cuda cores are available, else defaulting to the CPU.

In [60]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The first step is to prepare the data to be fed into the GNN. 

In [61]:
# Removes all categorical and string type columns to prepare data to be used as input into the GNN
# Since all the the numerical values have already been standardised for the KNN earlier, no further changesn need to be made
train_df = listing_df.drop(columns=['town', 'flat_type', 'street_name', 'storey_range','flat_model'])
X = train_df.drop(columns=['resale_price', 'month_index'])
y = data_df[['resale_price']]

x_arr = X.replace({True:1, False:0}).to_numpy()
y_arr = y.to_numpy()

node_ids = np.array(train_df.index)
id_to_pos = {nid: i for i, nid in enumerate(node_ids)}

# Edge tensor is constructed and assigned to the cpu
# A undirected, and unweighted graph is constructed to keep the modelling simple
edges = []
for u, v, data_w in G.edges(data=True):
    if u in id_to_pos and v in id_to_pos:
        edges.append((id_to_pos[u], id_to_pos[v]))
edges = np.array(edges, dtype=np.int64).T
# Make undirected explicitly
edge_tensor = np.hstack([edges, edges[::-1, :]])
edge_tensor = torch.tensor(edge_tensor, dtype=torch.long, device=device)

# Convert X and y to tensors and assign to CPU as well
x_tensor = torch.tensor(x_arr, dtype=torch.float32, device=device)
y_tensor = torch.tensor(y_arr, dtype=torch.float32, device=device)

# Split ids based on time; the train_mask will look at data before the month_index 24241
# this evaluates back to jan 2020
# This split gives an 80:20 split for train and test
train_mask = torch.tensor((listing_df.month_index<24241).values, device=device)
test_mask = torch.tensor((listing_df.month_index>=24241).values, device=device)

/tmp/ipykernel_7343/2194668649.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_arr = X.replace({True:1, False:0}).to_numpy()


The next step is to ready the data to be fed into the model. Since the size of the graph network is big, a loader will be used to break the graph into smaller batches. While this reduces the training speed, the memory impact is also reduced, making this training more accessible. The values in the loader can be changed to increase/reduce the batch size depending on the computational resources available.

In [62]:
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader

# Freezing seed to ensure reproducibiilty
torch.manual_seed(22)
np.random.seed(22)

data = Data(x=x_tensor, edge_index=edge_tensor, y=y_tensor, device=device)
data.train_mask, data.test_mask = train_mask, test_mask


train_idx = torch.where(data.train_mask)[0]
test_idx  = torch.where(data.test_mask)[0]

# Define loaders to reduce memory consumption
train_loader = NeighborLoader(
    data,
    num_neighbors=[15,10],
    input_nodes=train_idx,
    batch_size = 8192,
    shuffle = True
)

test_loader = NeighborLoader(
    data,
    num_neighbors=[-1,-1],
    input_nodes=test_idx,
    batch_size = 4096,
    shuffle = False
)

/home/soham/.local/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


The model itself is also defined. The SAGEConv allows to sample and aggregate the information present in the nodes neighbours, as well as higher degrees of connection, depending on the number of layers defined in the model. This method allows not just a single node's information to be used to predict the resale price, but also any nodes that - as previously defined - are considered to be close matches.

For this task, I've chosen a relatively light network with only 64 hidden nodes and 2 layers. If more compute is available, these hyper-parameters can be tuned to get better performance.

In [63]:
# Define GNN
from torch import nn
from torch_geometric.nn import SAGEConv

class price_model(nn.Module):
    def __init__(self, in_dim, hidden=64, layers=2):
        super().__init__()
        self.layer_list = []

        for i in range(layers):
            if i == 0:
                self.layer_list.append(SAGEConv(in_dim, hidden))
            else:
                self.layer_list.append(SAGEConv(hidden, hidden))

        self.head = nn.Sequential(
            nn.Linear(hidden, 32), nn.ReLU(), nn.Linear(32,1)
        )
    
    def forward(self, x, edge_index):
        # iterate through the layers for the forward pass
        for layer in self.layer_list:
            x = layer(x, edge_index)
            x = torch.relu(x)

        return self.head(x).squeeze(-1)

The last part would be to actually train a model and track how the loss decreases over time. An MSE objective is used to determine how well the model performs, as it's s continuous and differentiable loss function that handles regressions problems well.

While just a train and test dataset are used for this, with the test dataset acting as a validation set to determine an early stop, a combination of train, test and validation can be used if multiple hyper parameters or models are to be used to find the model that best reduces the objective function.

In [64]:
# init model as well as optimiser and loss function

model = price_model(in_dim = data.num_node_features)
opt = torch.optim.Adam(model.parameters(), lr=1e-1, weight_decay=1e-2)
loss_func = nn.MSELoss()

In [65]:
# define training step and evaluation function

def train(model, train_loader, opt, loss_func):
    model.train()
    total_loss = 0

    for batch in train_loader:
        batch = batch.to(device)
        
        opt.zero_grad()
        
        pred = model(batch.x, batch.edge_index)
        loss = loss_func(pred[:batch.batch_size], data.y[:batch.batch_size])
        loss.backward()
        
        opt.step()
        total_loss += loss.item()

    return total_loss/len(train_loader)

def eval(model, loss_func, loader):
    model.eval()

    total_mse = 0

    for batch in loader:
        with torch.no_grad():
            pred = model(batch.x, batch.edge_index)[:batch.batch_size]
            y = batch.y[:batch.batch_size]

            total_mse += loss_func(pred, y).item()

    return total_mse/len(loader)



As can be seen from the result of the training, the model is capable of optimising and predicting the resale price over time. An early stop is also evaluated at every 10th epoch to ensure that the model doesn't overfit on the training data.

In [30]:
# Training and evaluation flow
best_mse = float('inf')

for epoch in range(500):
    train(model, train_loader, opt, loss_func)

    if epoch%10 == 0:
        # train_mse = eval(model, loss_func, train_loader)
        test_mse = eval(model, loss_func, test_loader)
    
        print(f"Epoch{epoch} | Test RMSE {np.sqrt(test_mse):,.4f}")

        if test_mse < best_mse:
            best_mse = test_mse
        
        if test_mse > best_mse:
            break

        

Epoch0 | Test RMSE 466,430.9330
Epoch10 | Test RMSE 456,669.0031
Epoch20 | Test RMSE 423,921.2220
Epoch30 | Test RMSE 371,376.5566
Epoch40 | Test RMSE 308,402.0449
Epoch50 | Test RMSE 248,515.2676
Epoch60 | Test RMSE 205,814.0296
Epoch70 | Test RMSE 185,463.6202
Epoch80 | Test RMSE 179,347.9104
Epoch90 | Test RMSE 177,569.1015
Epoch100 | Test RMSE 176,076.8792
Epoch110 | Test RMSE 174,291.9357
Epoch120 | Test RMSE 172,411.3407
Epoch130 | Test RMSE 170,601.2078
Epoch140 | Test RMSE 168,936.5711
Epoch150 | Test RMSE 167,433.5058
Epoch160 | Test RMSE 166,089.8325
Epoch170 | Test RMSE 164,898.4979
Epoch180 | Test RMSE 163,845.9890
Epoch190 | Test RMSE 162,921.4957
Epoch200 | Test RMSE 162,106.6576
Epoch210 | Test RMSE 161,389.8800
Epoch220 | Test RMSE 160,759.4743
Epoch230 | Test RMSE 160,200.5447
Epoch240 | Test RMSE 159,703.1486
Epoch250 | Test RMSE 159,258.8177
Epoch260 | Test RMSE 158,858.1302
Epoch270 | Test RMSE 158,494.0311
Epoch280 | Test RMSE 158,160.2954
Epoch290 | Test RMSE 157,